In [1]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2

import IPython

IPython.Application.instance().kernel.do_shutdown(restart=True)

a
NumExpr defaulting to 2 threads.
⠏ Installing integrations...
  Using cached pyparsing-2.4.2-py2.py3-none-any.whl.metadata (1.1 kB)
Using cached pyparsing-2.4.2-py2.py3-none-any.whl (65 kB)
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 2.4.7
    Uninstalling pyparsing-2.4.7:
      Successfully uninstalled pyparsing-2.4.7


{'status': 'ok', 'restart': True}

In [1]:
NGROK_TOKEN = "2leOXvPgrpSgsENdT4wyHK87Rr3_3BwAkrcZSw2XoBZR3VVzz"

In [2]:
from zenml.environment import Environment
if Environment.in_google_colab():
  !pip install pyngrok
  !ngrok authtoken {NGROK_TOKEN}

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!rm -rf .zen
!zenml init

NumExpr defaulting to 2 threads.
Initializing the ZenML global configuration version to 0.65.0
Creating database tables
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
Creating default workspace 'default' ...
Creating default stack in workspace default...
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML reposi

In [4]:
from string import digits
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split


def train_test() -> None :
  digits = load_digits()
  data =  digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.2, shuffle=False)
  model = SVC(gamma=0.001)
  model.fit(X_train, y_train)
  test_acc = model.score(X_test, y_test)
  print(f"Test accuracy: {test_acc}")


train_test()

Test accuracy: 0.9583333333333334


In [5]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"],
]:

    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.2, shuffle=False)
    return X_train, X_test, y_train, y_test

@step
def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    model = SVC(gamma=0.001)
    model.fit(X_train, y_train)
    return model

@step
def evaluator(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin,
) -> float:
    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc}")
    return test_acc

In [6]:
from zenml import pipeline
@pipeline
def digits_pipeline():
    X_train, X_test, y_train, y_test = importer()
    model = svc_trainer(X_train=X_train, y_train=y_train)
    evaluator(X_test=X_test, y_test=y_test, model=model)

In [8]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Registered new pipeline: digits_pipeline.
Executing a new run.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Step importer has started.
Step importer has finished in 2.946s.
Step importer completed successfully.
Step svc_trainer has started.
Step svc_trainer has finished in 0.508s.
Step svc_trainer completed successfully.
Step evaluator has started.
Test accuracy: 0.9583333333333334
Step evaluator has finished in 1.008s.
Step evaluator completed successfully.
Pipeline run has finished in 4.874s.


In [ ]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
    if Environment.in_google_colab():
        from pyngrok import ngrok
        public_url = ngrok.connect(port)
        # Corrected ANSI escape codes for color formatting
        print(f"\033[31mIn Colab, use this URL instead: {public_url}\033[0m")
        !zenml up --blocking --port {port}
    else:
        !zenml up --port {port}

start_zenml_dashboard()


INFO:pyngrok.ngrok:Opening tunnel named: http-8237-c840d1ee-cea6-43e1-b67e-5103308f8f35


Opening tunnel named: http-8237-c840d1ee-cea6-43e1-b67e-5103308f8f35


INFO:pyngrok.process.ngrok:t=2024-09-07T04:24:41+0000 lvl=info msg="no configuration paths supplied"


t=2024-09-07T04:24:41+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2024-09-07T04:24:41+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2024-09-07T05:29:24+0000 lvl=info msg="join connections" obj=join id=70813966b4f7 l=127.0.0.1:8237 r=[2405:201:e00a:b034:fd95:738:4276:bc2f]:55412


INFO:pyngrok.process.ngrok:t=2024-09-07T05:29:24+0000 lvl=info msg="join connections" obj=join id=70219e71c346 l=127.0.0.1:8237 r=[2405:201:e00a:b034:fd95:738:4276:bc2f]:55412


t=2024-09-07T05:29:24+0000 lvl=info msg="join connections" obj=join id=70219e71c346 l=127.0.0.1:8237 r=[2405:201:e00a:b034:fd95:738:4276:bc2f]:55412


INFO:pyngrok.process.ngrok:t=2024-09-07T05:29:24+0000 lvl=info msg="join connections" obj=join id=b9329b215d20 l=127.0.0.1:8237 r=[2405:201:e00a:b034:fd95:738:4276:bc2f]:55412


t=2024-09-07T05:29:24+0000 lvl=info msg="join connections" obj=join id=b9329b215d20 l=127.0.0.1:8237 r=[2405:201:e00a:b034:fd95:738:4276:bc2f]:55412
